In [1]:
import torch

# CUDAが使えるかどうかを確認
cuda_available = torch.cuda.is_available()

if cuda_available:
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")
     

CUDA is available! GPU: Quadro RTX 5000


# **1枚画像**

In [1]:
from ultralytics import RTDETR
import wget
import cv2
import matplotlib.pyplot as plt

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model = RTDETR(r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt")

# 推論を実行: YOLOの推論
image_path = r'C:\Users\ykita\FacePhoto_instance\201-295\Image\293-20200311-6-095803_287b73c1d0b04ca725de2651547efd9a530231320625fb361bef3708dc3df9ee.jpg'
results = model(image_path, save=False)

for r in results:
    boxes = r.boxes  # YOLOv8の場合

    # YOLO形式出力
    for box in boxes:
        cls_id = int(box.cls[0])  # クラスID
        x_center, y_center, width, height = box.xywhn[0]

        # YOLO形式: class x_center y_center width height
        # 小数点以下6桁で出力したい場合
        print(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # 検出結果を画像に描画して表示
    # plt.figure(figsize=(12, 8))
    # plt.imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.show()

    # 結果の保存
    #cv2.imwrite('result.jpg', r.plot())

modelが未定義のため、ロードを実行します。


c:\Users\ykita\FacePhoto_instance\instanceenv\lib\site-packages\ultralytics\nn\tasks.py:634: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_locati


image 1/1 C:\Users\ykita\FacePhoto_instance\201-295\Image\293-20200311-6-095803_287b73c1d0b04ca725de2651547efd9a530231320625fb361bef3708dc3df9ee.jpg: 640x640 1 Right_eye, 1 Left_eye, 771.0ms
Speed: 9.3ms preprocess, 771.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
0 0.268958 0.487141 0.170173 0.112953
1 0.687053 0.477957 0.160687 0.102739


In [2]:
from ultralytics import RTDETR
import wget
import cv2
import matplotlib.pyplot as plt
import os

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model_path = r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt"
    model = RTDETR(model_path)

# 推論を実行
image_path = r'C:\Users\ykita\FacePhoto_instance\201-295\Image\293-20200311-6-095803_287b73c1d0b04ca725de2651547efd9a530231320625fb361bef3708dc3df9ee.jpg'
results = model(image_path, save=False)

# 保存先のテキストファイルパスを指定
save_txt_path = r'C:\Users\ykita\FacePhoto_instance\result_yolo.txt'

# YOLO形式の推論結果をテキストファイルに書き込み
with open(save_txt_path, 'w') as f:
    for r in results:
        boxes = r.boxes  # YOLOv8系の出力
        for box in boxes:
            cls_id = int(box.cls[0])  
            x_center, y_center, width, height = box.xywhn[0]
            # YOLO形式: class x_center y_center width height （小数点以下6桁）
            line = f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
            f.write(line)

print(f"YOLO形式の推論結果をテキストファイルに保存しました: {save_txt_path}")



image 1/1 C:\Users\ykita\FacePhoto_instance\201-295\Image\293-20200311-6-095803_287b73c1d0b04ca725de2651547efd9a530231320625fb361bef3708dc3df9ee.jpg: 640x640 1 Right_eye, 1 Left_eye, 879.9ms
Speed: 7.0ms preprocess, 879.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
YOLO形式の推論結果をテキストファイルに保存しました: C:\Users\ykita\FacePhoto_instance\result_yolo.txt


# **複数画像**

階層構造
```
archive.zip/
   ├── data.yaml  # configuration file
   ├── train.txt  # list of train subset image paths
   │
   ├── images/
   │   ├── train/  # directory with images for train subset
   │   │    ├── image1.jpg
   │   │    ├── image2.jpg
   │   │    ├── image3.jpg
   │   │    └── ...
   ├── labels/
   │   ├── train/  # directory with annotations for train subset
   │   │    ├── image1.txt
   │   │    ├── image2.txt
   │   │    ├── image3.txt
   │   │    └── ...

```

***.txt (ラベル)
```
0 0.260745 0.535950 0.218015 0.030600
1 0.760795 0.533850 0.213035 0.043800
```

daya.yaml
```
names:
  0: Right_eye
  1: Left_eye
path: .
train: train.txt
```

train.txt
```
data/images/train/***.jpg
data/images/train/***.jpg
...(pathの羅列。拡張子を合わせる)
```
※ラベルや画像と対応を合わせること



In [23]:
import os
import glob
import shutil
from tqdm import tqdm
from ultralytics import RTDETR

def create_yolo_dataset(orig_image_dir, output_dir, model_path):
    """
    YOLOデータセットの構造を作成する
    
    Args:
        orig_image_dir: 元の画像が存在するディレクトリ
        output_dir: 出力先のディレクトリ
        model_path: RTDETRモデルのパス
    """
    # 進捗バーの設定
    progress = tqdm(
        total=4,  
        desc="データセット作成",
        bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
    )

    # 必要なディレクトリを作成
    os.makedirs(os.path.join(output_dir, "labels", "train"), exist_ok=True)
    progress.update(1)
    
    # data.yamlの作成
    yaml_content = """names:
  0: Right_eye
  1: Left_eye
path: .
train: train.txt"""
    
    with open(os.path.join(output_dir, "data.yaml"), 'w', encoding='utf-8') as f:
        f.write(yaml_content)
    progress.update(1)
    
    # モデルのロード
    try:
        model = RTDETR(model_path)
    except Exception as e:
        print(f"モデルのロードに失敗: {e}")
        return
    
    # 画像ファイルのリストを取得
    image_files = []
    extensions = ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG', '*.png', '*.PNG', '*.tif', '*.TIF', '*.tiff', '*.TIFF']
    
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(orig_image_dir, ext)))
    
    total_files = len(image_files)
    print(f"\n合計 {total_files} 個のファイルが見つかりました")
    
    # 推論とラベル生成の進捗バー
    inference_progress = tqdm(
        total=total_files,
        desc="推論とラベル生成",
        bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
    )
    
    # train.txtの作成とラベルファイルの生成
    with open(os.path.join(output_dir, "train.txt"), 'w', encoding='utf-8') as f:
        for image_path in image_files:
            # 元の拡張子を保持
            basename = os.path.basename(image_path)
            name_without_ext = os.path.splitext(basename)[0]
            
            # 推論を実行
            results = model(image_path, save=False)
            
            # ラベルファイルのパス（train/配下に.txt）
            label_path = os.path.join(output_dir, "labels", "train", f"{name_without_ext}.txt")
            
            # YOLO形式の推論結果をテキストファイルに書き込み
            with open(label_path, 'w', encoding='utf-8') as lf:
                for r in results:
                    boxes = r.boxes
                    for box in boxes:
                        cls_id = int(box.cls[0])
                        x_center, y_center, width, height = box.xywhn[0]
                        line = f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
                        lf.write(line)
            
            # train.txtにYOLOフォーマットのパスを書き込み
            yolo_path = f"data/images/train/{basename}"
            f.write(f"{yolo_path}\n")
            inference_progress.update(1)
    
    inference_progress.close()
    progress.update(1)
    
    # ZIP圧縮
    print("\nZIP圧縮を開始します...")
    zip_path = output_dir + ".zip"
    total = sum(len(files) for _, _, files in os.walk(output_dir))
    
    with tqdm(total=total, desc="ZIP圧縮", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]') as pbar:
        for root, _, files in os.walk(output_dir):
            for file in files:
                pbar.update(1)
        shutil.make_archive(os.path.splitext(zip_path)[0], 'zip', output_dir)
    
    progress.update(1)
    progress.close()
    print(f"\nフォルダを {zip_path} に圧縮しました。")

# 使用例
if __name__ == "__main__":
    orig_image_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\image"
    output_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive"
    model_path = r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt"
    
    create_yolo_dataset(orig_image_dir, output_dir, model_path)

データセット作成:   0%|          | 0/4 [00:00<?]


合計 1390 個のファイルが見つかりました



image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\201-20071225-9-123502_024df9c564e854ac63c356338fae68d7e8ebd20fafde2856d07f854576d25bec.JPG: 640x640 1 Right_eye, 1 Left_eye, 717.6ms
Speed: 7.0ms preprocess, 717.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\201-20071225-9-123502_11ecdb1214665a16b19b0289b20c71d2c5a5a325f974312963bd044ddd10932f.JPG: 640x640 1 Right_eye, 1 Left_eye, 702.0ms
Speed: 4.0ms preprocess, 702.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\201-20071225-9-123502_50a5a27bf14844c0973f0c274e935b2766ef37d438492d264e490a1e2dcaf11b.JPG: 640x640 1 Right_eye, 1 Left_eye, 705.5ms
Speed: 6.0ms preprocess, 705.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


...

データセット作成:  75%|███████▌  | 3/4 [17:31<05:50]


ZIP圧縮を開始します...


データセット作成: 100%|██████████| 4/4 [17:34<00:00]


フォルダを C:\Users\ykita\FacePhoto_instance\inference\201-295\archive.zip に圧縮しました。


: 

In [18]:
import os
import glob
from ultralytics import RTDETR
from tqdm import tqdm  # tqdmライブラリをインポート

# 1) 画像のフォルダを指定する
orig_image_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\image"
# 2) 出力先のフォルダを指定する。フォルダがなければ作成する
dst_parent_dir= r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive" #####
dst_label_dir = os.path.join(dst_parent_dir, "labels", "train")
os.makedirs(dst_label_dir, exist_ok=True)

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model_path = r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt"
    model = RTDETR(model_path)

# 処理対象とする画像の拡張子
image_extensions = [
    '.jpg', '.jpeg', '.JPG', '.JPEG',
    '.png', '.PNG',
    '.tif', '.tiff', '.TIF', '.TIFF'
]

# 指定フォルダ配下の該当画像ファイルをすべて取得
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(orig_image_dir, f'*{ext}')))

# tqdmで進捗状況を表示しながらループ
for image_path in tqdm(image_files, desc='推論中', unit='枚'):
    # 推論を実行
    results = model(image_path, save=False)

    # 画像と同じファイル名（拡張子を除く）でテキストファイル出力
    basename = os.path.splitext(os.path.basename(image_path))[0]
    save_txt_path = os.path.join(dst_label_dir, f"{basename}.txt")

    # YOLO形式の推論結果をテキストファイルに書き込み
    with open(save_txt_path, 'w', encoding='utf-8') as f:
        for r in results:
            boxes = r.boxes  # YOLOv8系の出力
            for box in boxes:
                cls_id = int(box.cls[0])
                x_center, y_center, width, height = box.xywhn[0]
                # YOLO形式: class x_center y_center width height（小数点以下6桁）
                line = f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
                f.write(line)

print("すべての画像に対する推論が完了し、YOLO形式でラベルを出力しました。")


推論中:   0%|          | 0/1390 [00:00<?, ?枚/s]


image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\201-20071225-9-123502_024df9c564e854ac63c356338fae68d7e8ebd20fafde2856d07f854576d25bec.JPG: 640x640 1 Right_eye, 1 Left_eye, 1269.3ms
Speed: 42.0ms preprocess, 1269.3ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)


推論中:   0%|          | 1/1390 [00:01<32:57,  1.42s/枚]


image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\201-20071225-9-123502_11ecdb1214665a16b19b0289b20c71d2c5a5a325f974312963bd044ddd10932f.JPG: 640x640 1 Right_eye, 1 Left_eye, 784.6ms
Speed: 7.1ms preprocess, 784.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


推論中:   0%|          | 2/1390 [00:02<24:56,  1.08s/枚]

...

推論中:  84%|████████▎ | 1161/1390 [17:04<03:33,  1.07枚/s]


image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\265-20081224-5-155922_d2d60a3a244c4d8c5f2bab5612d66fcaea731565d19b0dd36472ebd94bf3d094.JPG: 640x640 1 Right_eye, 1 Left_eye, 732.4ms
Speed: 9.6ms preprocess, 732.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


推論中:  84%|████████▎ | 1162/1390 [17:05<03:22,  1.13枚/s]

推論中:  84%|████████▎ | 1162/1390 [17:05<03:21,  1.13枚/s]


KeyboardInterrupt: 

## **CVAT仕様のフォルダ構成に**

In [16]:
import os
import glob

# 現在のlabelsフォルダのパス
orig_image_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\image"
dst_parent_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive"


# yamlとimg_listのパスを定義
data_yaml_path = os.path.join(dst_parent_dir, "data.yaml")
img_list_path = os.path.join(dst_parent_dir, "train.txt")

# data.yaml を書き込み
with open(data_yaml_path, 'w', encoding='utf-8') as f:
    f.write("names:\n")
    f.write("  0: Right_eye\n")
    f.write("  1: Left_eye\n")
    f.write("path: .\n")
    f.write("train: train.txt\n")

# labels_dir配下の .txt ファイルから、img_list.txt を作成
txt_files = glob.glob(os.path.join(dst_label_dir, "*.txt"))

with open(img_list_path, 'w', encoding='utf-8') as f:
    for txt_file in txt_files:
        basename = os.path.splitext(os.path.basename(txt_file))[0]
        f.write(f"data/images/train/{basename}.jpg\n")

print("data.yaml と img_list.txt が、labelsフォルダと同じ階層に作成されました。")

data.yaml と img_list.txt が、labelsフォルダと同じ階層に作成されました。


In [17]:
import shutil

#dst_parent_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive"
# dst_parent_dirをzip圧縮
zip_path = dst_parent_dir + ".zip"
shutil.make_archive(dst_parent_dir, 'zip', dst_parent_dir)

print(f"フォルダを {zip_path} に圧縮しました。")


フォルダを C:\Users\ykita\FacePhoto_instance\inference\201-295\archive.zip に圧縮しました。
